In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import roc_curve, auc
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from torch.utils.data import WeightedRandomSampler

# Tải dữ liệu từ file CSV
duong_dan_file = 'C:\\Users\\phuoc\\Downloads\\NCKH\\financial_fraud_detection_dataset.csv'
try:
    df = pd.read_csv(duong_dan_file)
    print("Dữ liệu đã được tải thành công.")
except FileNotFoundError:
    print(f"Lỗi: Không tìm thấy file tại đường dẫn {duong_dan_file}. Vui lòng kiểm tra lại đường dẫn.")
    df = None

In [ ]:
if df is not None:
    print("\n5 dòng đầu tiên của dữ liệu:")
    display(df.head())

    print("\nThông tin về các cột và kiểu dữ liệu:")
    display(df.info())

    print("\nThống kê mô tả dữ liệu:")
    display(df.describe())

In [ ]:
print("Bắt đầu xử lý dữ liệu...")

# Tải lại dữ liệu để đảm bảo chúng ta bắt đầu với các cột gốc
duong_dan_file = 'C:\\Users\\phuoc\\Downloads\\Tai lieu thuc hanh Python\\IS23A AMD Lab\\Homework\\creditcard.csv'
try:
    df_processed = pd.read_csv(duong_dan_file)
    print("Dữ liệu đã được tải lại thành công.")
except FileNotFoundError:
    print(f"Lỗi: Không tìm thấy file tại đường dẫn {duong_dan_file}. Vui lòng kiểm tra lại đường dẫn.")
    df_processed = None

if df_processed is not None:
    # Loại bỏ cột 'Time'
    df_processed = df_processed.drop('Time', axis=1)
    print("Cột 'Time' đã được loại bỏ.")

    # Loại bỏ các hàng chứa giá trị NaN
    df_processed.dropna(inplace=True)
    print("Đã loại bỏ các hàng chứa giá trị NaN.")

    # Chuẩn hóa cột 'Amount'
    df_processed['NormalizedAmount'] = StandardScaler().fit_transform(df_processed['Amount'].values.reshape(-1, 1))
    print("Cột 'Amount' đã được chuẩn hóa thành 'NormalizedAmount'.")

    # Loại bỏ cột 'Amount' gốc
    df_processed = df_processed.drop('Amount', axis=1)
    print("Cột 'Amount' gốc đã được loại bỏ.")

    # Chia dữ liệu
    print("Bắt đầu chia dữ liệu thành tập huấn luyện và kiểm tra...")
    X = df_processed.drop('Class', axis=1).values
    y = df_processed['Class'].values
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    print("Dữ liệu đã được chia thành tập huấn luyện và kiểm tra.")

In [ ]:
# Chuyển đổi dữ liệu huấn luyện thành tensor PyTorch
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.float32)
print("Dữ liệu huấn luyện đã được chuyển đổi thành tensor.")

# Định nghĩa lớp dataset tùy chỉnh
class CustomDataset(Dataset):
    def __init__(self, features, targets):
        self.features = features
        self.targets = targets
        print("CustomDataset đã được khởi tạo.")

    def __len__(self):
        return len(self.features)

    def __getitem__(self, idx):
        return self.features[idx], self.targets[idx]
print("Lớp CustomDataset đã được định nghĩa.")

# Định nghĩa mô hình ODIN
class ODINModel(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(ODINModel, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size, output_size)
        # Loại bỏ hàm sigmoid tại đây
        print("Mô hình ODIN đã được khởi tạo.")

    def forward(self, x):
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        return x
print("Lớp ODINModel đã được định nghĩa.")

# Khởi tạo mô hình, hàm mất mát và trình tối ưu hóa
input_size = X_train.shape[1]
hidden_size = 64
output_size = 1
model = ODINModel(input_size, hidden_size, output_size)
# Sử dụng BCEWithLogitsLoss, hàm này ổn định hơn
criterion = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)
print("Mô hình, loss function và optimizer đã được khởi tạo.")

In [ ]:
# Vòng lặp huấn luyện
num_epochs = 5
batch_size = 64
print("Bắt đầu huấn luyện mô hình...")

# Tạo dataset và dataloader
train_dataset = CustomDataset(X_train_tensor, y_train_tensor)
print("Dataset huấn luyện đã được tạo.")
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
print("DataLoader huấn luyện đã được khởi tạo.")

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    print(f"Epoch {epoch+1}/{num_epochs} bắt đầu...")
    for inputs, labels in train_loader:
        optimizer.zero_grad()
        # Lấy logits từ mô hình
        outputs = model(inputs)
        # Sử dụng BCEWithLogitsLoss, hàm này yêu cầu logits
        loss = criterion(outputs, labels.unsqueeze(1))
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss/len(train_loader):.4f}")
print("Huấn luyện mô hình hoàn tất.")

### Giải thích codeblock 8: Huấn luyện mô hình

#### **Mục tiêu:**
- Huấn luyện mô hình trên tập dữ liệu huấn luyện.

#### **Quy trình:**
1. **Khởi tạo tham số huấn luyện:**
   - `num_epochs`: Số lần lặp qua toàn bộ dữ liệu huấn luyện.
   - `batch_size`: Kích thước của mỗi batch dữ liệu được sử dụng trong một lần huấn luyện.

2. **Tạo dataset và DataLoader:**
   - `CustomDataset`: Lớp dataset tùy chỉnh để quản lý dữ liệu huấn luyện.
   - `DataLoader`: Chia dữ liệu thành các batch và cung cấp cơ chế lặp qua dữ liệu.

3. **Vòng lặp huấn luyện:**
   - Tính toán đầu ra của mô hình.
   - Tính toán hàm mất mát (loss).
   - Lan truyền ngược (backpropagation) để cập nhật trọng số mô hình.

4. **In thông tin:**
   - Hiển thị tiến trình huấn luyện (epoch, loss trung bình).

#### **Kết quả:**
- Mô hình được huấn luyện xong, sẵn sàng để đánh giá.

#### **Tóm tắt:**
- **Mục tiêu:** Tập trung vào huấn luyện mô hình.
- **Kỹ thuật sử dụng:** Không sử dụng kỹ thuật ODIN, chỉ huấn luyện cơ bản.
- **Kết quả:** Mô hình được huấn luyện xong.

In [ ]:
# Đánh giá mô hình
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
model.eval()
# Lấy logits từ tập kiểm tra
logits = model(torch.tensor(X_test, dtype=torch.float32))
# Áp dụng hàm sigmoid để chuyển đổi logits thành xác suất
y_scores = torch.sigmoid(logits).detach().numpy()
# Chuyển đổi xác suất thành dự đoán nhị phân
y_pred = (y_scores > 0.5).astype(int)  # Chuyển đổi xác suất thành dự đoán nhị phân

# Tính toán đường cong ROC và AUC
fpr, tpr, _ = roc_curve(y_test, y_scores)
roc_auc = auc(fpr, tpr)

# Vẽ đường cong ROC
plt.figure()
plt.plot(fpr, tpr, color='darkorange', lw=2, label=f'Đường cong ROC (diện tích = {roc_auc:.2f})')
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
plt.xlabel('Tỷ lệ dương tính giả')
plt.ylabel('Tỷ lệ dương tính thật')
plt.title('Đường cong ROC')
plt.legend(loc='lower right')
plt.show()

print(f"Diện tích dưới đường cong (AUC): {roc_auc:.2f}")

# Ma trận nhầm lẫn
conf_matrix = confusion_matrix(y_test, y_pred)
print("Ma trận nhầm lẫn:")
print(conf_matrix)

# Báo cáo phân loại
class_report = classification_report(y_test, y_pred)
print("\nBáo cáo phân loại:")
print(class_report)

# Độ chính xác
accuracy = accuracy_score(y_test, y_pred)
print(f"\nĐộ chính xác: {accuracy:.2f}")

### Codeblock 9: Đánh giá mô hình với ODIN

#### **Mục tiêu:**
- Đánh giá hiệu suất của mô hình trên tập kiểm tra bằng cách sử dụng các kỹ thuật ODIN.

#### **Quy trình:**
1. **Thêm nhiễu đầu vào:**
   - Sử dụng gradient để tạo nhiễu (input perturbation).
2. **Áp dụng nhiệt độ scaling:**
   - Điều chỉnh logits bằng siêu tham số nhiệt độ.
3. **Tính toán xác suất:**
   - Chuyển đổi logits thành xác suất dự đoán.
4. **Đánh giá hiệu suất:**
   - Tính các chỉ số như ROC, AUC, ma trận nhầm lẫn, và báo cáo phân loại.
   - Vẽ biểu đồ ROC để trực quan hóa hiệu suất.

#### **Kết quả:**
- Các chỉ số đánh giá hiệu suất của mô hình với kỹ thuật ODIN được tính toán và trực quan hóa.

#### **Điểm khác biệt chính:**
- **Mục tiêu:**
  - Codeblock 9 tập trung vào đánh giá mô hình với kỹ thuật ODIN.
- **Kỹ thuật sử dụng:**
  - Sử dụng input perturbation và temperature scaling.
- **Kết quả:**
  - Hiệu suất mô hình được đánh giá với các chỉ số và biểu đồ ROC.

### Tại sao sử dụng BCEWithLogitsLoss và không cần hàm sigmoid ở lớp cuối cùng?

#### **Giới thiệu về BCEWithLogitsLoss:**
`BCEWithLogitsLoss` là một hàm mất mát đặc biệt được thiết kế cho các bài toán phân loại nhị phân. Điểm nổi bật của hàm này là:

1. **Chuyển đổi logits thành xác suất:**
   - Tự động áp dụng hàm sigmoid để chuyển đổi đầu ra (logits) của mô hình thành xác suất.
2. **Tính toán Binary Cross-Entropy:**
   - Sử dụng xác suất đã chuyển đổi để tính toán hàm mất mát.

#### **Vì sao không cần hàm sigmoid ở lớp cuối cùng?**
- Trong mô hình `ODINModel`, lớp cuối cùng (`fc2`) không sử dụng hàm sigmoid. Điều này là do `BCEWithLogitsLoss` đã bao gồm bước áp dụng sigmoid nội bộ.
- Nếu thêm sigmoid vào lớp cuối cùng, sẽ dẫn đến việc áp dụng sigmoid hai lần, gây ra:
  - **Lỗi tính toán.**
  - **Giảm độ chính xác của mô hình.**

#### **Lợi ích của việc không sử dụng sigmoid ở lớp cuối cùng:**
1. **Ổn định số học:**
   - Việc tính toán sigmoid và cross-entropy trong một bước giúp giảm thiểu các vấn đề về số học (như tràn số hoặc mất mát độ chính xác).
2. **Hiệu suất tốt hơn:**
   - Tránh tính toán sigmoid thủ công giúp tăng hiệu suất và đơn giản hóa mô hình.

#### **Tóm lại:**
- Mô hình không sử dụng hàm sigmoid ở lớp cuối cùng vì `BCEWithLogitsLoss` đã xử lý bước này.
- Khi cần xác suất đầu ra (như trong đánh giá hoặc dự đoán), bạn có thể áp dụng hàm sigmoid thủ công lên logits.

In [ ]:
# Các sửa đổi cụ thể cho phương pháp ODIN
import torch.nn.functional as F
import numpy as np
from torch.autograd import Variable

# Hàm áp dụng nhiệt độ scaling
def apply_temperature_scaling(logits, temperature):
    # Chia logits cho giá trị nhiệt độ để điều chỉnh
    return logits / temperature

# Hàm thêm nhiễu đầu vào
def add_input_perturbation(model, inputs, epsilon, temperature):
    # Đặt đầu vào thành biến có thể tính gradient
    inputs = Variable(inputs, requires_grad=True)
    # Tính toán logits từ mô hình
    logits = model(inputs)
    # Áp dụng nhiệt độ scaling cho logits
    scaled_logits = apply_temperature_scaling(logits, temperature)
    dummy_loss = torch.sum(logits)
    # Reset gradient của mô hình
    model.zero_grad()
    # Tính gradient của dummy loss đối với đầu vào
    dummy_loss.backward()
    gradient = inputs.grad.data
    # Thêm nhiễu vào đầu vào dựa trên dấu của gradient
    perturbed_inputs = inputs + epsilon * gradient.sign()
    return perturbed_inputs


# Đánh giá mô hình với các sửa đổi ODIN
temperature = 1000  # Siêu tham số cho nhiệt độ scaling
epsilon = 0.002  # Siêu tham số cho nhiễu đầu vào
model.eval()
# Thêm nhiễu vào tập kiểm tra
def add_input_perturbation_odin(model, inputs, epsilon):
    # Đặt đầu vào thành biến có thể tính gradient
    inputs = Variable(inputs, requires_grad=True)
    # Tính toán logits từ mô hình
    logits = model(inputs)
    dummy_loss = torch.sum(logits) # Tối đa hóa logit
    # Reset gradient của mô hình
    model.zero_grad()
    # Tính gradient của dummy loss đối với đầu vào
    dummy_loss.backward()
    gradient = inputs.grad.data
    # Thêm nhiễu vào đầu vào dựa trên dấu của gradient
    perturbed_inputs = inputs + epsilon * gradient.sign()
    return perturbed_inputs


# Đánh giá với ODIN
temperature = 1000  # Siêu tham số cho nhiệt độ scaling
epsilon = 0.002  # Siêu tham số cho nhiễu đầu vào

# Tính toán logits cho dữ liệu kiểm tra gốc
original_logits = model(torch.tensor(X_test, dtype=torch.float32))

# Thêm nhiễu vào dữ liệu kiểm tra gốc
perturbed_X_test_odin = add_input_perturbation_odin(model, torch.tensor(X_test, dtype=torch.float32), epsilon)

# Tính toán logits cho dữ liệu đã thêm nhiễu
perturbed_logits = model(perturbed_X_test_odin)

# Áp dụng nhiệt độ scaling cho logits của dữ liệu đã thêm nhiễu
scaled_perturbed_logits = apply_temperature_scaling(perturbed_logits, temperature)

# Chuyển đổi logits đã được scale thành xác suất (điểm ODIN)
odin_scores = torch.sigmoid(scaled_perturbed_logits).detach().numpy()

# Đối với các chỉ số đánh giá như ROC và AUC, chúng ta sử dụng điểm ODIN.
y_scores = odin_scores

# Chuyển đổi điểm ODIN thành dự đoán nhị phân (sử dụng ngưỡng, ví dụ: 0.5)
y_pred = (y_scores > 0.5).astype(int)

# Tính toán đường cong ROC và AUC
fpr, tpr, _ = roc_curve(y_test, y_scores)
roc_auc = auc(fpr, tpr)

# Vẽ đường cong ROC
plt.figure()
plt.plot(fpr, tpr, color='darkorange', lw=2, label=f'ROC curve with ODIN (area = {roc_auc:.2f})')
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
plt.xlabel('Tỷ lệ dương tính giả')
plt.ylabel('Tỷ lệ dương tính thật')
plt.title('Đường cong ROC với ODIN')
plt.legend(loc='lower right')
plt.show()

print(f"AUC with ODIN: {roc_auc:.2f}")

# Ma trận nhầm lẫn
conf_matrix = confusion_matrix(y_test, y_pred)
print("Ma trận nhầm lẫn với ODIN:")
print(conf_matrix)

# Báo cáo phân loại
class_report = classification_report(y_test, y_pred)
print("\nBáo cáo phân loại với ODIN:")
print(class_report)

# Độ chính xác
accuracy = accuracy_score(y_test, y_pred)
print(f"\nĐộ chính xác với ODIN: {accuracy:.2f}")